In [65]:
import matplotlib.pyplot as plt
import json
import requests
import numpy as np
import time
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
%matplotlib inline

In [5]:
def get_temp(elem):
    return elem['Measurements']['Temperature']['value']

def get_measure(elem, key):
    return elem['Measurements'][key]['value']

def get_timestamp(elem):
    return elem['Timestamp']
# Carbon Dioxide concentration

In [47]:
x = {'a' : 1, 'b' : 2}
for k in x.items():
    print(k)

('b', 2)
('a', 1)


In [6]:
int(round(time.time() * 1000))

1543056179239

In [64]:
URL_TEMPLATE = "https://apigtw.vaisala.com/hackjunction2018/saunameasurements/history?SensorID={}&before={}&after={}"
TIME_NOW = int(round(time.time() * 1000))

def load_to_file_json(sensor_id):
    now =   TIME_NOW
    start = 1542924742000
    delta = 3600000
    
    timestamps_all = []
    measures_all = defaultdict(list)
    for timestamp in range(start, now, delta):
        before = timestamp + delta
        after = timestamp
        response = requests.get(URL_TEMPLATE.format(sensor_id, before, after))
        json_array = json.loads(response.text)
        
        if len(json_array) > 0:
            timestamps_all.append([get_timestamp(x) for x in json_array])
            for item in json_array:
                for measure in item['Measurements'].items():
                    measures_all[measure[0]].append(measure[1]['value'])
    
    timestamps_all = np.concatenate(timestamps_all)
    df = pd.DataFrame()
    df['timestamps'] = timestamps_all
    for measure in measures_all.items():
        df[measure[0]] = measure[1]

    df.to_csv("data/{}.csv".format(sensor_id), index=False)

load_to_file_json("Bench1")

In [24]:
sensors = ['Ceiling1', 'Ceiling2', 'Stove1', 'Stove2',
'Bench1', 'Bench2', 'Bench3', 'Floor1', 'Doorway1', 'Outdoor1']

('Bench1', 'Bench2', 'Bench3', 'Floor1', 'Doorway1', 'Outdoor1')

In [ ]:
"Relative humidity"
"Carbon Dioxide concentration"
"Absolute humidity"
"Water vapor pressure"
"Enthalpy"
"Dew/frost point temperature"
"Wet-bulb temperature"
"Water concentration"
"Water vapor saturation pressure"
"Absolute humidity at NTP"